# Usage guidence

<font color='darkgreen' size=5>**newly trained model**</font>
1. **load model from config file with function `read_main_model_dataloader`  and get resumed model , val loader , popen from `out_dict`** 
  
```python
config_file = "/home/wergillius/Project/UTR_VAE/log/CrossStitch/CrossStitch_Model_MTL/DTP/gamma_t2_blr.ini" 
out_dict = read_main_model_dataloader(config_file,cuda_id=2) 
# from out_dict
model = out_dict['model'] 
train_loader = out_dict['train_loader'] 
val_loader = out_dict['val_loader'] 
POPEN = out_dict['popen'] 
```
2. **forward**   

```python
y_true_ls = []
y_pred_ls = []
cuda_id =2
```
    
```python
with torch.no_grad():  
    for X,Y in tqdm(val_loader):

        y_true_ls.append(Y.cpu().numpy())
        X = X.float().cuda(cuda_id)

        y_pred = model.backbone['RL'](X)

        y_pred_ls.append(y_pred.cpu().numpy())
        torch.cuda.empty_cache

# aggrate result
y_true_ay = np.stack([arry[:,0] for arry in y_true_ls])
y_pred_ay = np.stack(y_pred_ls).squeeze()
y_true_f = y_true_ay.flatten()
y_pred_f = y_pred_ay.flatten()
```

3. **save result**  

```python
save_result_to_csv(y_true_f,y_pred_f,save_path,motif_detection)
```    

4.**get 4 data for analysis**

```python
uAUG_tre,uAUG_pred,nAUG_tre,nAUG_pred = get_4_types_data_from_csv(df)
```

5.**compute R square**

```python
exp_r2 = compute_r2(y_true_f,y_pred_f)
```
6.**plot the result**

```python
my_kde_joint_plot(uAUG_tre,uAUG_pred,nAUG_tre,nAUG_pred,'plot_title')
```
<font color='darkgreen' size=5>**analysis from saved csv**</font>

1. **read csv**


2. **get 4 data for analysis**


3. **compute R square**


4. **plot the result**

# Analysis part

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import re
import PATH
import torch
from tqdm import tqdm

from importlib import reload
import analysis_for_MTL as A_MTL

from matplotlib import pyplot as plt


In [ ]:
import seaborn as sns

In [ ]:
reload(A_MTL)

## some function

all function are saved to `analysis_for_MTL.py`

In [ ]:
def pipe_line_for_MTL(config_file):
    config_file = "/home/wergillius/"+config_file
    out_dict = A_MTL.read_main_model_dataloader(config_file,cuda_id=2)

    model = out_dict['model']
    train_loader = out_dict['train_loader']
    val_loader = out_dict['val_loader']
    POPEN = out_dict['popen']

    y_true_ls = []
    y_pred_ls = []
    recon_acc_ls=[]
    cuda_id = 2

    with torch.no_grad():
        for X,Y in tqdm(val_loader):
            X = X.float().cuda(cuda_id)
            y_true_ls.append(Y.cpu().numpy())

            out = model(X)

            acc = compute_acc(X,out['Recons'][0])

            y_pred_ls.append(out['RL'])
            recon_acc_ls.append(acc)
            torch.cuda.empty_cache()

    # y_true_ls contain y_ture and motif annotation
    recon_acc_ay = np.stack([arry.detach().cpu().numpy() for arry in recon_acc_ls]).flatten()
    y_true_ay = np.stack([arry[:,0] for arry in y_true_ls])  
    y_pred_ay = np.stack([arry.detach().cpu().numpy() for arry in y_pred_ls]).squeeze()
    y_true_f = y_true_ay.flatten()
    y_pred_f = y_pred_ay.flatten()

    csv_path = 'some_plot/Cross_stitch/%s.csv'%POPEN.run_name
    df = A_MTL.save_result_to_csv(y_true_ls,y_true_f,y_pred_f,save_path=csv_path,motif_detection=True)

    df.loc[:,'recon_acc'] = recon_acc_ay
    df.to_csv(csv_path,index=False)
    del model
    return df

In [ ]:
def pipeline_for_RL_backbone(config_file,cuda_id=3):

    os.path.exists(config_file)
    baseline_out_dict = A_MTL.read_main_model_dataloader(config_file,cuda_id)

    model = baseline_out_dict['model']
    train_loader = baseline_out_dict['train_loader']
    val_loader = baseline_out_dict['val_loader']
    POPEN = baseline_out_dict['popen']

    y_true_ls = []
    y_pred_ls = []
    
    with torch.no_grad():
        for X,Y in tqdm(val_loader):
            X = X.float().cuda(cuda_id)
            y_true_ls.append(Y.cpu().numpy())

            y_pred = model(X)
            y_pred_ls.append(y_pred)

    with_uAUG = np.stack([ary[:,1] for ary in y_true_ls]).squeeze().flatten()
    y_true_f = np.stack([ary[:,0] for ary in y_true_ls]).squeeze().flatten()
    y_pred_f = np.stack([ts.detach().cpu().numpy() for ts in y_pred_ls]).flatten()
    
    r2 = A_MTL.compute_r2(y_true_f,y_pred_f)
    del model
    return A_MTL.save_result_to_csv(y_true_ls,y_true_f,y_pred_f,save_path='some_plot/backbone/%s_result.csv'%POPEN.run_name,motif_detection=False),r2

## Cross stitch model 

### <font color='darkred' size=5>read exp `gamma_t2_blr`</font>

In [ ]:
config_file = "Project/UTR_VAE/log/CrossStitch/CrossStitch_Model_MTL/DTP/gamma_t2_blr.ini"

gamma_b2_blr = pipe_line_for_MTL(config_file)

In [ ]:
# gamma_t2_blr = A_MTL.save_result_to_csv(y_true_ls,y_true_f,y_pred_f,save_path='some_plot/gamma_t2_blr_result.csv')

gamma_b2_blr_4 = A_MTL.get_4_types_data_from_csv(gamma_b2_blr)

gamma_t2_blr_r2 = A_MTL.compute_r2(gamma_b2_blr.Y_true.values,gamma_b2_blr.Y_pred.values)

In [ ]:
my_kde_joint_plot(*gamma_b2_blr_4,'Cross_stitch model with DTP\n'+r'$\alpha=0.9\ \beta=0.1\ lr=1e-3$'
                  ,r2=0.815,xlim=(0.7,10.3),ylim=(0.7,10.3))

### read model `a5b5_g2_11`

In [ ]:
config_file = "Project/UTR_VAE/log/CrossStitch/CrossStitch_Model_MTL/a5b5/a5b5_g2_11.ini"

a5b5_g2_11 = pipe_line_for_MTL(config_file)

In [ ]:
a5b5_g2_11_4 = A_MTL.get_4_types_data_from_csv(a5b5_g2_11)

a5b5_g2_11_r2 = A_MTL.compute_r2(a5b5_g2_11.Y_true.values,a5b5_g2_11.Y_pred.values)

In [ ]:
a5b5_g2_11_r2

In [ ]:
my_kde_joint_plot(*a5b5_g2_11_4,'Cross_stitch model with DTP\n alpha=0.5 bet=0.5 gamma 2 -1 1'
                  ,r2=0.841,xlim=(0.7,10.3),ylim=(0.7,10.3))

### read model `a5b5_g2_11_sslr`

In [ ]:
config_file = "Project/UTR_VAE/log/CrossStitch/CrossStitch_Model_MTL/a5b5/a5b5_g2_11_sslr.ini"

a5b5_g2_11_sslr = pipe_line_for_MTL(config_file)

In [ ]:
a5b5_g2_11_sslr_4 = A_MTL.get_4_types_data_from_csv(a5b5_g2_11_sslr)

a5b5_g2_11_sslr_r2 = A_MTL.compute_r2(a5b5_g2_11_sslr.Y_true.values,a5b5_g2_11_sslr.Y_pred.values)

In [ ]:
print('toal r2 : ',a5b5_g2_11_sslr_r2)

print('uAUG r2 : ',A_MTL.compute_r2(*a5b5_g2_11_sslr_4[:2]))

print('nAUG r2 : ',A_MTL.compute_r2(*a5b5_g2_11_sslr_4[2:]))

In [ ]:
a5b5_g2_11_sslr_4[0].mean()


In [ ]:
a5b5_g2_11_sslr_4[2].mean()

In [ ]:
my_kde_joint_plot(*a5b5_g2_11_sslr_4,'Cross_stitch model with DTP\n' + r'$\alpha$=0.5 $\beta$=0.5 $\gamma$ 2 -1 1 lr=1e-6',
                  r2='%.3f'%a5b5_g2_11_sslr_r2,xlim=(0.7,10.3),ylim=(0.7,10.3))

#### load form csv

In [ ]:
a5b5_g2_11_csv = pd.read_csv('some_plot/a5b5_g2_11_result.csv')

In [ ]:
a5b5_g2_11_csv.head(5)

## backbone model 

#### `RL_test`

In [ ]:
# backbone_RL_test = A_MTL.save_result_to_csv(y_true_ls,y_true_ay,y_pred_ay,save_path='some_plot/backbone_RL_test_result.csv',motif_detection=False)

backbone_RL_test = pd.read_csv("some_plot/backbone_RL_test_result.csv")

uAUG_tre,uAUG_pred,nAUG_tre,nAUG_pred = A_MTL.get_4_types_data_from_csv(backbone_RL_test)

backbone_RL_test_r2 = A_MTL.compute_r2(y_true_f,y_pred_f)


In [ ]:
my_kde_joint_plot(uAUG_tre,uAUG_pred,nAUG_tre,nAUG_pred,'Backbone Model')

#### L_regressor_MTL/test_run

In [ ]:
RL_test_l1_10epoch = "Project/UTR_VAE/log/Backbone/RL_regressor_MTL/test_run/RL_test_l1_10epoch.ini"
RL_test_l1 = "Project/UTR_VAE/log/Backbone/RL_regressor_MTL/test_run/RL_test_l1.ini"
RL_test_sl1 = "Project/UTR_VAE/log/Backbone/RL_regressor_MTL/test_run/RL_test_sl1.ini"

In [ ]:
for config_f in [RL_test_l1_10epoch,RL_test_l1,RL_test_sl1]:
    locals()[config_f.split('/')[-1].replace('.ini','_df')] = pipeline_for_RL_backbone(config_f)

In [ ]:
0.83226
0.83226
0.83226

In [ ]:
RL_test_l1_10epoch_4 = A_MTL.get_4_types_data_from_csv(RL_test_l1_10epoch_df)
RL_test_l1_4 = A_MTL.get_4_types_data_from_csv(RL_test_l1_df)
RL_test_sl1_4 = A_MTL.get_4_types_data_from_csv(RL_test_sl1_df)

In [ ]:
my_kde_joint_plot(*RL_test_l1_10epoch_4,'RL_test_l1_10epoch',r2=0.83226,xlim=(1,10),ylim=(1,10))
my_kde_joint_plot(*RL_test_l1_4,'RL_test_l1',r2=0.83226,xlim=(1,10),ylim=(1,10))
my_kde_joint_plot(*RL_test_sl1_4,'RL_test_sl1',r2=0.83226,xlim=(1,10),ylim=(1,10))

### RL_regressor_MTL/vali_by_deepest

In [ ]:
config_dir = "/home/wergillius/Project/UTR_VAE/log/Backbone/RL_regressor_MTL/vali_by_deepest"

In [ ]:
config_files = [os.path.join(config_dir,file) for file in os.listdir(config_dir) if file.split('.')[-1]=='ini']
exp_name = [file.split('.ini')[0] for file in os.listdir(config_dir)  if '.ini' in file]

In [ ]:
result_dir = "/home/wergillius/Project/UTR_VAE/ipynb/some_plot/backbone/vali_by_deepest/"
exp_name = [file.split('.csv')[0] for file in os.listdir(result_dir)]
df_ls = {}

In [ ]:
exp_name

In [ ]:
for csv in os.listdir(result_dir):
    df_ls[csv.split('.csv')[0]] = pd.read_csv(os.path.join(result_dir,csv))

In [ ]:
data_of_4_dict = {exp:A_MTL.get_4_types_data_from_csv(df_ls[exp]) for exp in exp_name}

In [ ]:
for exp in exp_name:
    my_kde_joint_plot(*data_of_4_dict[exp],exp,xlim=(0.7,10.3),ylim=(0.7,10.3))

### RL_regressor_MTL/split_two_csv

In [ ]:
config_dir = "/home/wergillius/Project/UTR_VAE/log/Backbone/RL_regressor_MTL/human_vlength_dataset"

In [ ]:
config_file = [os.path.join(config_dir,file) for file in os.listdir(config_dir) if '.ini' in file]
exp_name = [file.split('.ini')[0] for file in os.listdir(config_dir) if '.ini' in file]

In [ ]:
two_csv_dict = {}
two_csv_r2_dict = {}
for exp in exp_name:
    two_csv_dict[exp],two_csv_r2_dict[exp] = pipeline_for_RL_backbone(os.path.join(config_dir,exp+'.ini'),cuda_id=1)

In [ ]:
x = torch.randn((1,2)).cuda()

map them back to 1-10

In [ ]:
for exp in exp_name[1:]:
    for Y in ['Y_true','Y_pred']:
        two_csv_dict[exp].loc[:,Y] = scaler.inverse_transform(two_csv_dict[exp].loc[:,Y].values.reshape(-1,1))

In [ ]:
# recalcualte the r2 on the original space
for exp in exp_name[1:]:
    two_csv_r2_dict[exp] = A_MTL.compute_r2(two_csv_dict[exp].Y_true.values,two_csv_dict[exp].Y_pred.values)

extract uAUG_true uAUG_pred nAUG_true nAUG_pred.   

and plot them

In [ ]:
two_csv_4_dict = {exp:A_MTL.get_4_types_data_from_csv(two_csv_dict[exp]) for exp in exp_name}

In [ ]:
for exp in exp_name:
    A_MTL.my_kde_joint_plot(*two_csv_4_dict[exp],title=exp,r2="%.4f"%two_csv_r2_dict[exp],text_posi=(7,2),xlim = (1.3,10),ylim = (1.3,10))

### scaler to reverse the scaling

In [ ]:
from sklearn import preprocessing

In [ ]:
val_csv = pd.read_csv('/data/users/wergillius/UTR_VAE/multi_task/val_split_like_paper.csv')

val_rl = val_csv.rl.values

scaler = preprocessing.StandardScaler().fit(val_rl.reshape(-1,1))

### RL_regressor_MTL/split_two_csv_deep

In [ ]:
log_dir="/home/wergillius/Project/UTR_VAE/log/Backbone/RL_regressor_MTL/split_two_csv_deep"
exp_name = [file.split('.ini')[0] for file in os.listdir(log_dir) if '.ini' in file]

In [ ]:
sptc_deep_dict = {}
sptc_d_r2_dict = {}

for exp in exp_name:
    config = os.path.join(log_dir,exp+'.ini')
    sptc_deep_dict[exp] , sptc_d_r2_dict[exp] = pipeline_for_RL_backbone(config)

In [ ]:
scaled_exp = [exp for exp in exp_name if 'unscaled' not in exp]

In [ ]:
scaled_exp

map them back to 1-10

In [ ]:
for exp in scaled_exp:
    for Y in ['Y_true','Y_pred']:
        sptc_deep_dict[exp].loc[:,Y] = scaler.inverse_transform(sptc_deep_dict[exp].loc[:,Y].values.reshape(-1,1))

In [ ]:
# recalcualte the r2 on the original space
for exp in scaled_exp:
    sptc_d_r2_dict[exp] = A_MTL.compute_r2(sptc_deep_dict[exp].Y_true.values,sptc_deep_dict[exp].Y_pred.values)

extract uAUG_true uAUG_pred nAUG_true nAUG_pred.   

and plot them

In [ ]:
sptc_d_4data_dict = {exp:A_MTL.get_4_types_data_from_csv(sptc_deep_dict[exp]) for exp in exp_name}

In [ ]:
for exp in exp_name:
    
    text_posi=(7,2)
    xlim = (1.3,10)
    ylim = (1.3,10)
    
    A_MTL.my_kde_joint_plot(*sptc_d_4data_dict[exp],title=exp,r2="%.4f"%sptc_d_r2_dict[exp],text_posi=text_posi,xlim=xlim,ylim=ylim)

### human vlength dataset

In [ ]:
config_dir="/home/wergillius/Project/UTR_VAE/log/Backbone/RL_regressor_MTL/human_vlength_dataset"

In [ ]:
exp_name = [file.split('.ini')[0] for file in os.listdir(config_dir) if '.ini' in file]

In [ ]:
# hv : human_vlength 
hv_df_dict = {}
hv_r2_dict = {}

for exp in exp_name:
    # all experiments are trained with scaled data
    hv_df_dict[exp],hv_r2_dict[exp] = pipeline_for_RL_backbone(os.path.join(config_dir,exp+'.ini'),cuda_id=0)

In [ ]:
for exp in exp_name:
    for Y in ['Y_true','Y_pred']:
        hv_df_dict[exp].loc[:,Y] = scaler.inverse_transform(hv_df_dict[exp].loc[:,Y].values.reshape(-1,1))

In [ ]:
# recalcualte the r2 on the original space
for exp in exp_name:
    hv_r2_dict[exp] = A_MTL.compute_r2(hv_df_dict[exp].Y_true.values,hv_df_dict[exp].Y_pred.values)

extract uAUG_true uAUG_pred nAUG_true nAUG_pred.   

and plot them

In [ ]:
sptc_d_4data_dict = {exp:A_MTL.get_4_types_data_from_csv(hv_df_dict[exp]) for exp in exp_name}

In [ ]:
for exp in exp_name:
    
    text_posi=(7,2)
    xlim = (1.3,10)
    ylim = (1.3,10)
    
    A_MTL.my_kde_joint_plot(*sptc_d_4data_dict[exp],title=exp,r2="%.4f"%hv_r2_dict[exp],text_posi=text_posi,xlim=xlim,ylim=ylim)

## Baseline model

### L_50

In [ ]:
config_dir = '/home/wergillius/Project/UTR_VAE/log/Baseline/Baseline_MTL/L_50/'

In [ ]:
exp_name = [file.split('.ini')[0] for file in os.listdir(config_dir) if '.ini' in file]

In [ ]:
L50_df_dict ={}
L50_r2_dict ={}
for exp in exp_name:
    L50_df_dict[exp],L50_r2_dict[exp] = pipeline_for_RL_backbone(os.path.join(config_dir,exp+'.ini'))

In [ ]:
exp_name

In [ ]:
for exp in exp_name:
    for yy in ['Y_true','Y_pred']:
        L50_df_dict[exp].loc[:,yy] = scaler.inverse_transform(L50_df_dict[exp].loc[:,yy].values.reshape(-1,1))
    L50_r2_dict[exp] = A_MTL.compute_r2(L50_df_dict[exp].Y_true.values,L50_df_dict[exp].Y_pred.values)

In [ ]:
L50_data4_dict = {exp:A_MTL.get_4_types_data_from_csv(L50_df_dict[exp]) for exp in exp_name}

In [ ]:
for exp in exp_name:
    A_MTL.my_kde_joint_plot(*L50_data4_dict[exp],r2='%.4f'%L50_r2_dict[exp],title=exp,xlim=(1,10),ylim=(0,9))

### load exp `TO_SEQ_TE : reg_w05_05`

#### from starch

In [ ]:
# backbone_RL_test = A_MTL.save_result_to_csv(y_true_ls,y_true_ay,y_pred_ay,save_path='some_plot/backbone_RL_test_result.csv',motif_detection=False)

backbone_RL_test = pd.read_csv("some_plot/backbone_RL_test_result.csv")

uAUG_tre,uAUG_pred,nAUG_tre,nAUG_pred = A_MTL.get_4_types_data_from_csv(backbone_RL_test)

backbone_RL_test_r2 = A_MTL.compute_r2(y_true_f,y_pred_f)

In [ ]:
model = baseline_out_dict['model']
train_loader = baseline_out_dict['train_loader']
val_loader = baseline_out_dict['val_loader']
POPEN = baseline_out_dict['popen']

In [ ]:
y_true_ls = []
y_pred_ls = []
recon_acc_ls=[]
cuda_id = 2

In [ ]:
with torch.no_grad():
    for X,Y in tqdm(val_loader):
        X = X.float().cuda(cuda_id)
        y_true_ls.append(Y.cpu().numpy())
        
        y_pred = model(X)
        
        acc = compute_acc(X,y_pred[1])
        
        y_pred_ls.append(y_pred)
        recon_acc_ls.append(acc)

In [ ]:
recon_acc_ay = torch.stack(recon_acc_ls).flatten().detach().cpu().numpy()

In [ ]:
# y_true_ls contain y_ture and motif annotation

recon_acc_ay = torch.stack(recon_acc_ls)

y_true_ay = np.stack([arry[:,0] for arry in y_true_ls])  
y_pred_ay = np.stack([arry[2].detach().cpu().numpy() for arry in y_pred_ls]).squeeze()
y_true_f = y_true_ay.flatten()
y_pred_f = y_pred_ay.flatten()

```python
TO_SEQ_TE_reg = A_MTL.save_result_to_csv(y_true_ls,y_true_f,y_pred_f,save_path='some_plot/TO_SEQ_TE_reg_w05_05.csv',motif_detection=False)
TO_SEQ_TE_reg.loc[:,'recon_acc'] = recon_acc_ay
TO_SEQ_TE_reg.to_csv('some_plot/TO_SEQ_TE_reg_w05_05.csv',index=False)
```

In [ ]:
TO_SEQ_TE_reg= pd.read_csv('some_plot/TO_SEQ_TE_reg_w05_05.csv')

#### resume
<font size=5 color='darkred'>**start from here**

In [ ]:
TO_SEQ_TE_reg = pd.read_csv('some_plot/TO_SEQ_TE_reg_w05_05.csv')

TO_SEQ_TE_reg_4 = A_MTL.get_4_types_data_from_csv(TO_SEQ_TE_reg)

TO_SEQ_TE_reg_r2 = A_MTL.compute_r2(y_true_f,y_pred_f)

In [ ]:
print('toal r2 : ',TO_SEQ_TE_reg_r2)

print('uAUG r2 : ',A_MTL.compute_r2(*TO_SEQ_TE_reg_4[:2]))

print('nAUG r2 : ',A_MTL.compute_r2(*TO_SEQ_TE_reg_4[2:]))

In [ ]:
my_kde_joint_plot(*TO_SEQ_TE_reg_4,'TO_SEQ_TE reg \n weight 0.5 : 0.5',xlim=(-3.2,1.8),ylim=(-3.2,1.8))

## plot reconstruction accuracy 

In [ ]:
recon_acc_of_4model = np.stack([TO_SEQ_TE_reg.recon_acc,gamma_b2_blr.recon_acc.values/1.01,a5b5_g2_11.recon_acc.values/1.01,a5b5_g2_11_sslr.recon_acc.values/1.01])

In [ ]:
recon_acc_of_4model[:2].shape

In [ ]:
plt.rcParams['font.weight'] = 'normal'
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['axes.labelpad'] = 5
plt.rcParams['axes.linewidth']= 2
plt.rcParams['xtick.labelsize']= 14
plt.rcParams['ytick.labelsize']= 14
plt.rcParams['xtick.major.size'] = 6
plt.rcParams['ytick.major.size'] = 6
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.minor.width'] = 1
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['axes.edgecolor'] = 'black'


In [ ]:
fig,ax=plt.subplots(figsize=(9,11))
sns.boxplot(data=recon_acc_of_4model.T,palette='vlag',width=0.7,linewidth=3);
ax.set_ylabel('Reconstruction Accuracy (%)',fontsize=20)
ax.set_xticklabels(['Hard_share',r'DTP_$\alpha 9 \beta 1$',r'DTP_$\alpha 5 \beta 5$',r'DTP_$\alpha 5 \beta 5$_slr'],fontsize=19);

# check generation

## 
<font color='navy' size=5>**visualize cross stitch unit**</font>

### Motif detection

In [ ]:
def cross_unit_plot(p,w):
    plt.figure(figsize=(w,3))
    plt.imshow(p,aspect='auto',cmap=cm.Greens);
    plt.yticks([0,1,2],model.tasks);

In [ ]:
Mo_param = []
for stage in model.stage:
    Mo_param.append(np.stack([next(model.cross_stitch_link[stage].param_dict['Motif'][task].parameters()).cpu().detach().numpy() for task in model.tasks]))

In [ ]:
cross_unit_plot(Mo_param[0],6)

cross_unit_plot(Mo_param[1],12)

cross_unit_plot(Mo_param[2],24)

cross_unit_plot(Mo_param[3],6)

### RL regression

In [ ]:
def cross_unit_plot(p,w):
    plt.figure(figsize=(w,3))
    plt.imshow(p,aspect='auto',cmap=cm.Oranges);
    plt.yticks([0,1,2],model.tasks);

In [ ]:
RL_param = []
for stage in model.stage:
    RL_param.append(np.stack([next(model.cross_stitch_link[stage].param_dict['RL'][task].parameters()).cpu().detach().numpy() for task in model.tasks]))

In [ ]:
cross_unit_plot(RL_param[0],6)

cross_unit_plot(RL_param[1],12)

cross_unit_plot(RL_param[2],24)

cross_unit_plot(RL_param[3],6)

### Recons

In [ ]:
def cross_unit_plot(p,w):
    plt.figure(figsize=(w,3))
    plt.imshow(p,aspect='auto',cmap=cm.Blues);
    plt.yticks([0,1,2],model.tasks);

In [ ]:
Recons_param = []
for stage in model.stage:
    Recons_param.append(np.stack([next(model.cross_stitch_link[stage].param_dict['Recons'][task].parameters()).cpu().detach().numpy() for task in model.tasks]))

In [ ]:
cross_unit_plot(Recons_param[0],6)

cross_unit_plot(Recons_param[1],12)

cross_unit_plot(Recons_param[2],24)

cross_unit_plot(Recons_param[3],6)

In [ ]:
test_xx = torch.rand((1,2,1)).cuda(3)

In [ ]:
torch.cuda.empty_cache()

## 
<font color='darkgreen' size=5>**get the code from soft share**</font>

In [ ]:
train_iter = iter(train_loader)

In [ ]:
RL = Y[:,0]

In [ ]:
sns.kdeplot(RL.numpy(),linewidth=4);

<font color='Orange' size=5>***encode***

In [ ]:
X_ls = []
taskfeature_ls  = []
Y_ls = []
cuda_id=2

In [ ]:
for i in range(20):
    X,Y = next(train_iter)
    X_ls.append(X.cpu().numpy())
    Y_ls.append(Y.cpu().numpy())
    X = X.float().cuda(cuda_id)
    with torch.no_grad():
        X = X.transpose(1,2) if X.shape[2] == 4 else X
        task_feature = {t:X for t in model.tasks}
        for stage in model.stage:
            task_feature = model.forward_stage(task_feature,stage)
        taskfeature_ls.append(task_feature)
        torch.cuda.empty_cache()

In [ ]:
RL_feature_ls = np.concatenate([feature['RL'].detach().cpu().numpy() for feature in taskfeature_ls],axis=0)
RL_feature_f = RL_feature_ls.reshape(2000,-1)

In [ ]:
rl_label_ls = np.concatenate([y[:,0] for y in Y_ls])

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

rl_pca = PCA(n_components=2)

In [ ]:
rl_tsne = TSNE(n_components=2,perplexity=20,verbose=1)

In [ ]:
RL_feature_tsne = rl_tsne.fit_transform(RL_feature_f)

In [ ]:
np.quantile(rl_label_ls,[0.25,0.75])

In [ ]:
np.quantile(rl_label_ls,[0.25,0.75])

In [ ]:
plt.scatter(RL_feature_tsne[:,0],RL_feature_tsne[:,1],s=2,c=rl_label_ls);

In [ ]:
rl_pca = PCA(n_components=2)

In [ ]:
RL_feature_pca = rl_pca.fit_transform(RL_feature_f)

In [ ]:
rl_pca.fit(RL_feature_f)

In [ ]:
print('hallo')

In [ ]:
rl_pca.transform()

In [ ]:
out,mu,sigma = model.backbone['Recons'].forward_tower(task_feature['Recons'])